In [ ]:
import pathlib
import requests

import graphviz
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets, metrics, model_selection
from sklearn import pipeline, preprocessing, tree

# Decision Trees

[Decision Trees](https://scikit-learn.org/stable/modules/tree.html) are a non-parametric supervised learning method used for [classification](https://scikit-learn.org/stable/modules/tree.html#tree-classification) and [regression](https://scikit-learn.org/stable/modules/tree.html#tree-regression). The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features. A tree can be seen as a piecewise constant approximation.

## Training and Visualizing a Decision Tree

In [ ]:
iris_data = datasets.load_iris(
    as_frame=True
)

In [ ]:
features_df = iris_data.data
target = iris_data.target

In [ ]:
features_df.head()

In [ ]:
target

In [ ]:
classifier = tree.DecisionTreeClassifier(max_depth=2, random_state=42)
_ = classifier.fit(features_df, target)

In [ ]:
_ = tree.export_graphviz(
    classifier,
    out_file="iris-tree.dot",
    feature_names=features_df.columns,
    class_names=iris_data.target_names,
    rounded=True,
    filled=True
)

In [ ]:
(
    graphviz.Source
            .from_file("iris-tree.dot")
)

In [ ]:
help(classifier.tree_)

## Making Predictions

In [ ]:
classifier.predict(features_df)

## Predicting Class Probabilities

In [ ]:
classifier.predict_proba(features_df)

## Regularization

In [ ]:
tree.DecisionTreeClassifier?

In [ ]:
metrics.get_scorer_names()

In [ ]:
classifier = tree.DecisionTreeClassifier()
_ = classifier.fit(features_df, target)
metrics.accuracy_score(
    target,
    classifier.predict(features_df),
)

In [ ]:
classifier = tree.DecisionTreeClassifier()
cv_scores = model_selection.cross_val_score(
    classifier,
    features_df,
    target,
    cv=5,
    n_jobs=-1,
    scoring="accuracy"
)

In [ ]:
cv_scores

In [ ]:
np.mean(cv_scores)

## Exercise

Fit a decision tree classifier to the following dataset. Select an appropriate scoring metric and evaluate the performance of your classifier using cross-validation. Is your classifier under-fitting? Over-fitting? Tune the regularization hyperparameters to improve the performance of your classifier.

In [ ]:
datasets.load_breast_cancer?

In [ ]:
breast_cancer_data = datasets.load_breast_cancer(
    as_frame=True
)

In [ ]:
print(breast_cancer_data.DESCR)

### Solution

First, we fit and evaluate a default decision tree classifier.

In [ ]:
features_df = breast_cancer_data.data
target = breast_cancer_data.target

In [ ]:
classifier = tree.DecisionTreeClassifier()
_ = classifier.fit(features_df, target)

metrics.recall_score(
    target,
    classifier.predict(features_df)
)

In [ ]:
# default classifier with no-regularization
classifier = tree.DecisionTreeClassifier()
cv_scores = model_selection.cross_val_score(
    classifier,
    features_df,
    target,
    cv=5,
    n_jobs=-1,
    scoring="recall"
)
print(np.mean(cv_scores))

Looks like the default decision tree classifier, without any regularization is over-fitting and we need to regularize. Code in the cell below can be used to experiment with different regularization strategies through "hand-tuning" individual hyper-parameters.

In [ ]:
classifier = tree.DecisionTreeClassifier(
    max_depth=None,
    min_samples_leaf=0.04,
    min_samples_split=2,
)
cv_scores = model_selection.cross_val_score(
    classifier,
    features_df,
    target,
    cv=5,
    n_jobs=-1,
    scoring="recall"
)
print(np.mean(cv_scores))

In [ ]:
classifier = tree.DecisionTreeClassifier(
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=0.04,
)
_ = classifier.fit(features_df, target)

_ = tree.export_graphviz(
    classifier,
    out_file="breast-cancer-tree.dot",
    feature_names=features_df.columns,
    class_names=breast_cancer_data.target_names,
    rounded=True,
    filled=True
)

_ = (
    graphviz.Source
            .from_file("breast-cancer-tree.dot")
)

Next, we can use grid search to automate the process of hyper-parameter tuning for our decision tree.

In [ ]:
classifier = tree.DecisionTreeClassifier()

tuned_decision_tree = model_selection.GridSearchCV(
    classifier,
    param_grid=[
      {
          "max_depth": [1, 2, 4, None],
          "criterion": ["gini", "entropy"],
          "min_samples_split": [2, 0.01, 0.04, 0.16],
          "min_samples_leaf": [1, 0.01, 0.04, 0.16],
      },
    ],
    cv=5,
    scoring="recall",
    n_jobs=-1,
)

In [ ]:
_ = tuned_decision_tree.fit(
    features_df,
    target,
)

In [ ]:
tuned_decision_tree.best_params_

In [ ]:
tuned_decision_tree.best_score_

In [ ]:
_ = tree.export_graphviz(
    tuned_decision_tree.best_estimator_,
    out_file="best-breast-cancer-tree.dot",
    feature_names=features_df.columns,
    class_names=breast_cancer_data.target_names,
    rounded=True,
    filled=True
)

_ = (
    graphviz.Source
            .from_file("best-breast-cancer-tree.dot")
)

## Understanding Feature Importance

One of the nice features of decision trees is that they provide a way to measure the importance of each of feature. Understanding feature importance is a topic all unto itself. If you are interested in pulling this thread, then I recommend that you start with [SHapley Additive Explanations (SHAP)](https://shap.readthedocs.io/en/latest/index.html) and then take a look through [*Interpretable Machine Learning*](https://christophm.github.io/interpretable-ml-book/).

In [ ]:
classifier = tree.DecisionTreeClassifier()
_ = classifier.fit(features_df, target)

In [ ]:
pd.Series(
    tuned_decision_tree.best_estimator_.feature_importances_,
    index=breast_cancer_data.data.columns
).sort_values(
    ascending=False
)

## Exercise

Grow a forest by following these steps:

1. Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 80% of the training instances selected randomly. Hint: you can use Scikit-Learn’s ShuffleSplit class for this.

2. Train one decision tree on each subset, using the best hyperparameter values found in the previous exercise. Evaluate these 1,000 decision trees on the test set. Since they were trained on smaller sets, these decision trees will likely perform worse than the first decision tree, achieving only about 80% accuracy.


3. Now comes the magic. For each test set instance, generate the predictions of the 1,000 decision trees, and keep only the most frequent prediction (you can use SciPy’s mode() function for this). This approach gives you majority-vote predictions over the test set.

4. Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model. Congratulations, you have trained a random forest classifier!